In [1]:
import sys
sys.path.append('../api/src')

import re
import os

from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
)
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_GPT_MODEL = "gpt-4-1106-preview"


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())


mp_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


tools = [
    MPThermoExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)),
     arxiv,
     wikipedia,
]

/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [2]:


prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template


"MPThermoExpert: MPThermoExpert(input: str) - Theromodynamics expert that has access to Materials Project thermo endpoint, args: {'input': {'title': 'Input', 'description': 'Complete question to ask the assistatn agent. Should include all the context and details needed to answer the question holistically.', 'type': 'string'}}\nMPElasticityExpert: MPElasticityExpert(input: str) - Elasticity expert that has access to Materials Project elasticity endpoint, including bulk, shear, and young's modulus, poisson ratio, and universal anisotropy index, args: {'input': {'title': 'Input', 'description': 'Complete question to ask the assistatn agent. Should include all the context and details needed to answer the question holistically.', 'type': 'string'}}\nMPDielectricExpert: MPDielectricExpert(input: str) - Dielectric expert that has access to Materials Project dielectric endpoint, args: {'input': {'title': 'Input', 'description': 'Complete question to ask the assistatn agent. Should include all 

In [3]:
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)
model = ChatOpenAI(temperature=0, streaming=True, max_retries=5,
                   model=OPENAI_GPT_MODEL, api_key=OPENAI_API_KEY)
agent = create_openai_tools_agent(model.with_config(
    {'tags': ['react-multi-input-json']}), tools, prompt)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config({
    'run_name': 'react-multi-input-json',
})

In [4]:
chunks = []

async for chunk in agent_executor.astream(
    {"input": "What's the Bulk Modulus Fe?"},
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(
                f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `MPElasticityExpert` with input `{'input': 'What is the bulk modulus of iron (Fe)?'}`
---


> Entering new AgentExecutor chain...


/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/Users/oliver/code/llamp/venv/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Fe",
    "fields": "material_id,formula_pretty,bulk_modulus"
  }
}
```Action:
```
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "Fe",
    "fields": "material_id,formula_pretty,bulk_modulus"
  }
}
```{"formula": "Fe", "fields": "material_id,formula_pretty,bulk_modulus"}


Retrieving ElasticityDoc documents: 100%|██████████| 4/4 [00:00<00:00, 66576.25it/s]


[{'formula_pretty': 'Fe', 'material_id': 'mp-136', 'elastic_tensor': {'raw': [[533.1049561765178, 179.90963906823504, 155.99496312976856, 9.714451465470118e-15, -1.2989609388114328e-14, 2.7755575615628906e-15], [179.90963906823504, 527.5690949703794, 153.63137470367502, 9.547918011776345e-15, -1.3933298959045711e-14, 2.8474848866768495e-14], [155.99496312976856, 153.63137470367502, 624.2810588699431, 7.327471962526033e-15, -1.3544720900426911e-14, 5.551115123125764e-16], [9.714451465470118e-15, 9.547918011776345e-15, 7.327471962526033e-15, 168.93500235013536, 0.0, 2.187656762018417e-30], [-1.2989609388114328e-14, -1.3933298959045711e-14, -1.3544720900426911e-14, 0.0, 168.42688381862257, -4.37996423834296e-15], [2.7755575615628906e-15, 2.8474848866768495e-14, 5.551115123125764e-16, 2.187656762018417e-30, -4.37996423834296e-15, 171.94797365144507]], 'ieee_format': [[529.0, 182.0, 155.0, -0.0, -0.0, 0.0], [182.0, 529.0, 155.0, -0.0, -0.0, 0.0], [155.0, 155.0, 624.0, 0.0, 0.0, 0.0], [-0.0,